# Analisando as Cotas Parlamentares dos Anos de 2019, 2020, 2021, 2022

In [271]:
import pandas as pd 

In [272]:
#Abrindo os arquivos csv e transformando em dataframes
df_22 = pd.read_csv('dados/despesa_ceaps_2022.csv', sep=';',encoding= 'latin-1',skiprows=1)
df_21 = pd.read_csv('dados/despesa_ceaps_2021.csv', sep=';',encoding= 'latin-1',skiprows=1)
df_20 = pd.read_csv('dados/despesa_ceaps_2020.csv', sep=';',encoding= 'latin-1',skiprows=1)
df_19 = pd.read_csv('dados/despesa_ceaps_2019.csv', sep=';',encoding= 'latin-1',skiprows=1)

#confirmando se os nomes das colunas são iguais
eh_equal = all(df.columns.equals(outro_df.columns) for outro_df in [df_22, df_21, df_20, df_19])
eh_equal

True

In [273]:
#Realizando a concatenação de todos os dfs em um único arquivo. 
df = pd.concat([pd.read_csv(file, sep=';', encoding='latin-1',skiprows=1) for file in ['dados/despesa_ceaps_2019.csv',
                                                            'dados/despesa_ceaps_2020.csv',
                                                            'dados/despesa_ceaps_2021.csv',
                                                            'dados/despesa_ceaps_2022.csv']],ignore_index=True)

df.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582758,04/01/2019,Despesa com pagamento de energia elétrica do e...,"66,02",2116543
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582755,04/01/2019,Despesa com pagamento de energia elétrica do e...,"139,98",2116546
2,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,00119,07/01/2019,Despesa com pagamento de aluguel de imóvel par...,6000,2113817
3,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.423.963/0001-11,OI MÓVEL S.A.,86161151,25/12/2018,Despesa com pagamento de telefonia para o escr...,"316,39",2116541
4,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007236036,04/02/2019,Despesa com pagamento de energia elétrica para...,"99,45",2116550


In [274]:
#obtendo o tamanho do dataframe
print('Dimensões do Data Frame: ', df.shape)

Dimensões do Data Frame:  (69333, 11)


In [275]:
# obtendo informações sobre as colunas 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69333 entries, 0 to 69332
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ANO                69333 non-null  int64 
 1   MES                69333 non-null  int64 
 2   SENADOR            69333 non-null  object
 3   TIPO_DESPESA       69333 non-null  object
 4   CNPJ_CPF           69333 non-null  object
 5   FORNECEDOR         69333 non-null  object
 6   DOCUMENTO          66370 non-null  object
 7   DATA               69333 non-null  object
 8   DETALHAMENTO       38583 non-null  object
 9   VALOR_REEMBOLSADO  69333 non-null  object
 10  COD_DOCUMENTO      69333 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 5.8+ MB


### Conversão do Tipo de dado
Considerando as informações das colunas, podemos converter os atributos VALOR_REEMBOLSADO e DATA para tipos de dados mais apropriados. 

In [276]:
df['VALOR_REEMBOLSADO']= df.VALOR_REEMBOLSADO.str.replace(',', '.') #troca a vírgula por ponto em toda series
df['VALOR_REEMBOLSADO'] = pd.to_numeric(df.VALOR_REEMBOLSADO, downcast='float') #converte a string para o tipo numérico float64
df['VALOR_REEMBOLSADO'].dtype

dtype('float64')

In [277]:
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y', errors='coerce')
df['DATA'].dtype

dtype('<M8[ns]')

In [278]:
df['VALOR_REEMBOLSADO'].describe()

count     69333.000000
mean       1413.131098
std        3098.174309
min           0.010000
25%         158.140000
50%         426.040000
75%        1570.560000
max      120000.000000
Name: VALOR_REEMBOLSADO, dtype: float64

In [279]:
df_selecionado = df[pd.isna(df['DATA'])]
df_selecionado

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
10986,2019,1,LÚCIA VÂNIA,"Aluguel de imóveis para escritório político, c...",08.573.731/0001-38,LDE Contabilidade e Administração de Condomíni...,0107007,NaT,Taxa condominial,271.26,2114251
31996,2020,5,PLÍNIO VALÉRIO,Divulgação da atividade parlamentar,27.209.437/0001-96,Excelsior Comunicação Digital Ltda,2020/9,NaT,NaN,1150.00,2144535
37872,2021,11,CARLOS VIANA,"Passagens aéreas, aquáticas e terrestres nacio...",16.978.175/0001-08,ADRIA VIAGENS E TURISMO LTDA,YK1QHD,NaT,"Companhia Aérea: AZUL, Localizador: YK1QHD. Pa...",339.90,2169237


In [280]:
df_selecionado['VALOR_REEMBOLSADO'].mean()

587.0533333333333

Considerando que temos apenas três registros com datas preenchidas de forma incorreta e que a média do VALORE_REEMBOLSADO destes registros é inferior à media da amostra e próximo à mediana, optei por eliminar essas insconsistências. 

In [282]:
df.dropna(subset=['DATA'], inplace=True)

In [283]:
df.shape

(69330, 11)